<a href="https://colab.research.google.com/github/SaraBabakN/Model-Search/blob/master/Correlation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


train_set = torchvision.datasets.CIFAR10('./dataset', train=True, download=True,
                                         transform=transforms.Compose([transforms.ToTensor()]))


def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
lr = 0.01
batch_size = 200
dataset_size = 50000
epoch_num = 40
cor_ths = 0.65
data_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)


In [ ]:
class myModel(nn.Module):
    def __init__(self, block,groups=1, width_per_group=64):
        super(myModel, self).__init__()
        norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer
        self.inplanes = 64
        self.dilation = 1
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3,64, kernel_size=7, stride=2, padding=3,bias=False)
        self.bn1 = norm_layer(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64)
        self.layer2 = self._make_layer(block, 64)
        self.layer3 = self._make_layer(block, 128, stride=2)
        self.layer4 = self._make_layer(block, 128,)
        self.layer5 = self._make_layer(block, 256, stride=2)
        self.layer6 = self._make_layer(block, 256)
        self.layer7 = self._make_layer(block, 512, stride=2)
        self.layer8 = self._make_layer(block, 512)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , 10)

    def _make_layer(self, block, planes, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes, stride),
                norm_layer(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes
        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)
class BasicBlock(nn.Module): 
    expansion = 1 
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        norm_layer = nn.BatchNorm2d
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out    

def conv1x1(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

def myResnet(pretrained=False, progress=True, **kwargs):
    return myModel(BasicBlock,**kwargs)

In [ ]:
myNet = myResnet()
optimizer = optim.Adam(myNet.parameters(), lr=lr)

In [ ]:
correct = 0 
for batch in data_loader:
    images, labels = batch
    images = images.cuda() 
    labels = labels.cuda()
    preds = myNet(images)
    correct = correct + get_num_correct(preds,labels)
print("accuracy" , correct/dataset_size)

accuracy 0.9836


In [ ]:
layers = [] 
layers.append(myNet.conv1.weight)
layers.append(list(myNet.layer1.modules())[2].weight)
layers.append(list(myNet.layer1.modules())[5].weight)
layers.append(list(myNet.layer2.modules())[2].weight)
layers.append(list(myNet.layer2.modules())[5].weight)
layers.append(list(myNet.layer3.modules())[2].weight)
layers.append(list(myNet.layer3.modules())[5].weight)
layers.append(list(myNet.layer4.modules())[2].weight)
layers.append(list(myNet.layer4.modules())[5].weight)
layers.append(list(myNet.layer5.modules())[2].weight)
layers.append(list(myNet.layer5.modules())[5].weight)
layers.append(list(myNet.layer6.modules())[2].weight)
layers.append(list(myNet.layer6.modules())[5].weight)
layers.append(list(myNet.layer7.modules())[2].weight)
layers.append(list(myNet.layer7.modules())[5].weight)
layers.append(list(myNet.layer8.modules())[2].weight)
layers.append(list(myNet.layer8.modules())[5].weight)

In [ ]:
counter = 0 
for num in range (len(layers)): 
  output= [] 
  convLayer = layers[num]
  layer = convLayer.clone().detach().cpu()
  num_filter = layer.shape[0] 
  layer = layer.reshape( num_filter , -1 )
  for i in range( num_filter - 2 ):
      for j in range ( i + 1  , num_filter):
        corr = np.corrcoef(layer[i],layer[j])
        if ( abs(corr[0,1]) > cor_ths ):
          output.append([i,j])
  print(len(output)/num_filter/(num_filter-1) * 200)
  for couple in output:
    (layers[num])[couple[1]] = 0 
    counter = counter + 1 
print(counter)    

In [ ]:
correct = 0 
for batch in data_loader:
    images, labels = batch
    images = images.cuda() 
    labels = labels.cuda()
    preds = myNet(images)
    correct = correct + get_num_correct(preds,labels)
print("accuracy" , correct/dataset_size)

accuracy 0.95176


In [ ]:
importance_matrices = [] 
for layer in layers: 
  x = torch.norm(layer , dim=(1,2,3) , p = 1 ) 
  x = x / sum(x)
  importance_matrices.append(x)


In [ ]:
i = 0 
counter = 0 
for mat in importance_matrices: 
  plt.figure(i)
  t= range(mat.shape[0])
  plt.plot(t , mat[t].clone().detach().cpu() , "*")
  
  plt.show 
  i = i + 1 

In [ ]:
batch = next(iter(data_loader))
images, labels = batch
images = images.cuda() 
labels = labels.cuda()
x = myNet.conv1(images)
x = myNet.bn1(x)
x = myNet.relu(x)
x = myNet.maxpool(x)
x = myNet.layer1(x)
x = myNet.layer2(x)
l3 = myNet.layer3(x)
l4 = myNet.layer4(l3)
l5 = myNet.layer5(l4)
l6= myNet.layer6(l5)
l7 = myNet.layer7(l6) 
l8 = myNet.layer8(l7) 


In [ ]:
modulList = list(myNet.layer3.modules())
# step 1
x1 = modulList[2](x)
x1 = modulList[3](x1)
x1 = modulList[4](x1)
x1 = modulList[5](x1)
x1 = modulList[6](x1)
#step 2 
x2 = modulList[7](x) 

modulList = list(myNet.layer5.modules())
# Step 1
x5 = modulList[2](l4)
x5 = modulList[3](x5)
x5 = modulList[4](x5)
x5 = modulList[5](x5)
x5 = modulList[6](x5)
# Step 2
x6 = modulList[7](l4)

modulList = list(myNet.layer7.modules())
#Step 1 
x9 = modulList[2](l6)
x9 = modulList[3](x9)
x9 = modulList[4](x9)
x9 = modulList[5](x9)
x9 = modulList[6](x9)
#Step 2
x10 = modulList[7](l6)

In [ ]:
x1_0 = x1[0].clone().detach().cpu()
x2_0 = x2[0].clone().detach().cpu() 
x1_0[1].shape 


torch.Size([4, 4])

In [ ]:
output= []
x1_0 = x1.clone().detach().cpu()
x2_0 = x2.clone().detach().cpu() 
x5_0 = x5.clone().detach().cpu()
x6_0 = x6.clone().detach().cpu()
x9_0 = x9.clone().detach().cpu()
x10_0 = x10.clone().detach().cpu()

for filter in range(2):
  corr = np.corrcoef(x1_0[filter].reshape(-1) , x2_0[filter].reshape(-1))
  print(corr.shape)
  if ( corr[0,1] > cor_ths ):
    print("1" , filter)
  corr = np.corrcoef(x5_0[filter].reshape(-1) , x6_0[filter].reshape(-1))
  if ( corr[0,1] > cor_ths ):
    print("2" , filter)
  corr = np.corrcoef(x9_0[filter].reshape(-1) , x10_0[filter].reshape(-1))
  if ( corr[0,1] > cor_ths ):
    print("3" , filter)


# for filter in range(x5_0.shape[0]):
#   corr = np.corrcoef(x5_0[filter].reshape(-1) , x6_0[filter].reshape(-1))
#   if ( corr[0,1] > cor_ths ):
#     output.append(filter)
# print(len(output)/x5_0.shape[0] * 100)   



# output= []
# for filter in range(x9_0.shape[0]):
#   corr = np.corrcoef(x9_0[filter].reshape(-1) , x10_0[filter].reshape(-1))
#   if ( corr[0,1] > cor_ths ):
#     output.append(filter)
# print(len(output)/x10_0.shape[0] * 100)   



# x11 = nn.ReLU(inplace=True)(x9)
# x12 = nn.ReLU(inplace=True)(x10)
# x9_0 = x9[0].clone().detach().cpu()
# x10_0 = x12[0].clone().detach().cpu()
# output= []
# for filter in range(x9_0.shape[0]):
#   corr = np.corrcoef(x9_0[filter].reshape(-1) , x10_0[filter].reshape(-1))
#   if ( corr[0,1] > cor_ths ):
#     output.append(filter)
# print(len(output)/x10_0.shape[0] * 100)   



(2, 2)
(2, 2)


In [ ]:
output= []
for i in range (200):
  y9 = x9[i].clone().detach().cpu()
  y10 = x10[i].clone().detach().cpu()
  corr = np.corrcoef(abs(y9.reshape(-1)) , abs(y10.reshape(-1)))
  if ( corr[0,1] > cor_ths ):
    output.append([i])
  print(corr[0,1])
# print(len(output)/x10_0.shape[0] * 100)   
